# **4G Network Performance**

**Contexto:** 
Você é um engenheiro de performance de redes móveis e precisa avaliar dados de tráfego das estações para determinar se indicador está dentro do limite desejado. 

**Condições do Problema:**
1. Você precisa carregar os dados de performance, identificar as diferentes estações e avaliar se os indicadores de acessibilidade RRC e RAB estão dentro dos limiares 99,8% e 99,9% de eficiência respectivamente
2. Caso uma estação esteja com um dos indicadores fora da meta, essa informação deverá ser carregada em outro arquivo que servirá de suporte para identificar os casos a serem tratados para melhoria de qualidade.

In [2]:
import pandas as pd

## Extract

In [3]:
# Carrega o CSV ignorando as primeiras 7 linhas que são cabeçalho do relatório sem informação útil
# A linha 8 automaticamente se torna o cabeçalho
df = pd.read_csv('Followup_LTE_4G00.csv', skiprows=7)
df.index = range(1, len(df) + 1)

Coluna com nome das estações:
<ul>
<li>NE Name</li>
</ul>

Colunas com indicadores de performance de interesse:
<ul>
<li>L.RRC.ConnReq.Succ (None)</li>
<li>L.RRC.ConnReq.Att (None)</li>
<li>L.E-RAB.AttEst (None)</li>
<li>L.E-RAB.SuccEst (None)</li>
</ul>

## Transform

In [40]:
#variáveis de limite de performance para cada indicador
limite_rrc = 0.998;
limite_rab = 0.999;

In [42]:
# Remove espaços em branco
df.columns = df.columns.str.strip()
df['NE Name'] = df['NE Name'].str.strip()

In [44]:
# Lista com contadores de performance a serem coletados e avaliados
contadores_performance = [
    'L.RRC.ConnReq.Succ (None)',
    'L.RRC.ConnReq.Att (None)',
    'L.E-RAB.AttEst (None)',
    'L.E-RAB.SuccEst (None)']

In [46]:
# Agrupa apenas colunas com contadores de interesse para cada estação
df_contadores = df.groupby('NE Name')[contadores_performance].sum()

In [48]:
#print(df_contadores.dtypes)

In [50]:
Efic_RRC = df_contadores['L.RRC.ConnReq.Succ (None)'] / df_contadores['L.RRC.ConnReq.Att (None)']

In [52]:
Efic_RAB = (df_contadores['L.E-RAB.SuccEst (None)'] / df_contadores['L.E-RAB.AttEst (None)'])

In [54]:
#print(Efic_RRC)
#print(Efic_RAB)

In [56]:
kpi_site = pd.DataFrame({
    'NE Name': df_contadores.index,
    'Efic_RRC': Efic_RRC.values,
    'Efic_RAB': Efic_RAB.values
})

In [58]:
#kpi_site

In [60]:
# Verifica se há eficiências abaixo de 0.999
kpis_afetados = []

In [62]:
for index, row in kpi_site.iterrows():
    ne_name = row['NE Name']
    efic_rrc = row['Efic_RRC']
    efic_rab = row['Efic_RAB']
    
    if efic_rrc < limite_rrc:
        kpis_afetados.append({
            'NE Name': ne_name,
            'KPI': 'Efic_RRC',
            'Valor': efic_rrc
        })
    
    if efic_rab < limite_rab:
        kpis_afetados.append({
            'NE Name': ne_name,
            'KPI': 'Efic_RAB',
            'Valor': efic_rab
        })

## Load

Salva o resultado em um novo CSV para indicar as estações com performance degradada a serem atuadas


In [66]:
# Cria dataframe com os problemas encontrados e salva em .csv
if kpis_afetados:
    df_kpis_afetados = pd.DataFrame(kpis_afetados)
    print("KPI fora da meta:")
    print(df_kpis_afetados)

    # Salva em arquivo CSV
    df_kpis_afetados.to_csv('Worst_Sites.csv', index=False, encoding='utf-8')
    print("\nArquivo 'Worst_Sites.csv' salvo com sucesso!")
else:
    print("Nenhum KPI fora da meta.")

KPI fora da meta:
    NE Name       KPI     Valor
0  MCOC1_CE  Efic_RAB  0.998763

Arquivo 'Worst_Sites.csv' salvo com sucesso!
